### Politeness features and Markers in Convokit

In [1]:
# Note that I ran this using Python 3.9
import convokit
import spacy
import pandas as pd
import re

2023-05-23 14:00:34.130936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from convokit import Corpus, Speaker, Utterance
from convokit import download
from convokit import TextParser

In [3]:
from convokit import PolitenessStrategies

ps = PolitenessStrategies()
full_jury_data = pd.read_csv('../../data/raw_data/jury_conversations_with_outcome_var.csv')
spacy_nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [4]:
full_jury_data.head()

,batch_num,round_num,speaker_hash,speaker_nickname,timestamp,message,majority_pct,num_flipped,flipped_pct,num_votes
0,0,0,5e7e1e0031f4e454e196c30b,niceRhino,2020-04-20T18:27:20.125Z,Hello!,1.0,1,0.333333,3
1,0,0,5e31d6e4e31c5304c46f1413,culturedCow,2020-04-20T18:27:23.764Z,Hi!,1.0,1,0.333333,3
2,0,0,5e7e4f4c31f4e454e196c9c4,spryBison,2020-04-20T18:27:27.724Z,Hello,1.0,1,0.333333,3
3,0,0,5d482ea421c9be351f762255,youngLion,2020-04-20T18:27:30.410Z,Hi,1.0,1,0.333333,3
4,0,0,5e84cc3c50f6e364321d6265,smallGiraffe,2020-04-20T18:27:35.506Z,hi,1.0,1,0.333333,3


In [11]:
def get_politeness_stragies(text):
    utt = ps.transform_utterance(
        text, spacy_nlp=spacy_nlp
    )
    return(utt.meta["politeness_strategies"])

In [6]:
# Apply transform function and convert to dataframe
transformed_df = full_jury_data.head(100)['message'].apply(get_politeness_stragies).apply(pd.Series)

# Concatenate the transformed dataframe with the original dataframe
df_featurized = pd.concat([full_jury_data.head(100), transformed_df], axis=1)

In [7]:
df_featurized

,batch_num,round_num,speaker_hash,speaker_nickname,timestamp,message,majority_pct,num_flipped,flipped_pct,num_votes,...,feature_politeness_==1st_person_start==,feature_politeness_==2nd_person==,feature_politeness_==2nd_person_start==,feature_politeness_==Indirect_(greeting)==,feature_politeness_==Direct_question==,feature_politeness_==Direct_start==,feature_politeness_==HASPOSITIVE==,feature_politeness_==HASNEGATIVE==,feature_politeness_==SUBJUNCTIVE==,feature_politeness_==INDICATIVE==
0,0,0,5e7e1e0031f4e454e196c30b,niceRhino,2020-04-20T18:27:20.125Z,Hello!,1.000000,1,0.333333,3,...,0,0,0,1,0,0,0,0,0,0
1,0,0,5e31d6e4e31c5304c46f1413,culturedCow,2020-04-20T18:27:23.764Z,Hi!,1.000000,1,0.333333,3,...,0,0,0,1,0,0,0,0,0,0
2,0,0,5e7e4f4c31f4e454e196c9c4,spryBison,2020-04-20T18:27:27.724Z,Hello,1.000000,1,0.333333,3,...,0,0,0,1,0,0,0,0,0,0
3,0,0,5d482ea421c9be351f762255,youngLion,2020-04-20T18:27:30.410Z,Hi,1.000000,1,0.333333,3,...,0,0,0,1,0,0,0,0,0,0
4,0,0,5e84cc3c50f6e364321d6265,smallGiraffe,2020-04-20T18:27:35.506Z,hi,1.000000,1,0.333333,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,2,5e7e4f4c31f4e454e196c9c4,newLion,2020-04-20T19:03:53.219Z,"Tha's true, she ins't considering her husband ...",0.600000,0,0.000000,5,...,0,0,0,0,0,0,0,0,0,0
96,0,2,5e84cc3c50f6e364321d6265,likelyRabbit,2020-04-20T19:04:02.642Z,I don't agree. Not the asshole. She's too busy.,0.600000,0,0.000000,5,...,1,0,0,0,0,0,0,0,0,0
97,1,0,5e9de5be9dc0406f8987226c,culturedPanda,2020-04-20T18:35:51.231Z,"That person is definitely an asshole, I honest...",0.666667,1,0.166667,6,...,0,0,0,0,0,0,0,0,0,0
98,1,0,5e9de4e29dc0406f89872254,newHippo,2020-04-20T18:36:08.493Z,This guy doesn't seem to understand the delica...,0.666667,1,0.166667,6,...,0,0,0,0,0,0,1,1,0,0
